# $k$-means clustering

[$k$-means clustering](https://www.wikiwand.com/en/K-means_clustering) is one of the simpler clustering algorithms. The idea is that we want to represent each cluster by a vector who's coordinates are obtained by averaging the coordinates of observations that belong to it. An observation belongs to the cluster it is closest to. The "closest to" part depends on a metric provided by the user.

This is a bit of a [chicken or the egg](https://www.wikiwand.com/en/Chicken_or_the_egg) kind of problem. On the one hand we want to know to which cluster each observation belongs to. On the other hand the position of each cluster depends on the observations that are assigned to it.

## Let's look at some penguins 🐧

In [5]:
import pandas as pd

penguins = pd.read_csv('https://raw.githubusercontent.com/mcnakhaee/palmerpenguins/master/palmerpenguins/data/penguins.csv')
penguins = penguins.dropna()
penguins.sample(5)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
28,Adelie,Biscoe,37.9,18.6,172.0,3150.0,female,2007
303,Chinstrap,Dream,49.5,19.0,200.0,3800.0,male,2008
196,Gentoo,Biscoe,50.5,15.9,222.0,5550.0,male,2008
274,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,female,2009
252,Gentoo,Biscoe,48.5,15.0,219.0,4850.0,female,2009


In [7]:
import altair as alt

(
    alt.Chart(penguins)
    .mark_circle()
    .encode(
        x='flipper_length_mm',
        y='bill_length_mm',
        color=alt.Color('species', scale=alt.Scale(scheme='dark2')),
    )
    .interactive()
)


alt.Chart(...)

$k$-means operates on distances between observations. As for all distance based methods, it is important to standardize the data before applying the algorithm. Otherwise, variables with large values will dominate the distance computations.

In [9]:
from sklearn import preprocessing

features = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].copy()
features.loc[:] = preprocessing.StandardScaler().fit_transform(features)
species = penguins['species']


## Explanation

A collection of points $\{x_i\}$, for $i = 1,\ldots n$, where $x_i\in \mathbb R^d$. 

The goal is to find $k$ *centers*, $\{\mu_\ell\}$, $\ell = 1,\ldots k$, and assign each point $x$ to a *cluster* $C_\ell$ with center $\mu_\ell$, as to minimize the *total intra-cluster distance* 
$$
\arg\min_{C} \sum_{\ell=1}^k \sum_{x_i \in C_\ell} \| x_i - \mu_\ell\|^2. 
$$
Here, $\mu_\ell$ is the mean of points in $C_\ell$. The total intra-cluster distance is the total squared Euclidean distance from each point to the center of its cluster. It's a measure of the variance or internal coherence of the clusters. 


It turns that finding the solution to the K-means method is [NP-hard](https://www.wikiwand.com/en/NP-hardness). This doesn't mean we can't find an approximate (and good enough) solution. The most popular approximate method is [Lloyd's algorithm](https://www.wikiwand.com/en/Lloyd%27s_algorithm), and it isn't just used for K-means.

Lloyd's algorithm goes as follows:

1. Generate random centroids (a fancy name for the center of each cluster) 
2. Assign each observation to the closest centroid.
3. Move each centroid to the average of the observations that are assigned to it.
4. Repeat from step 2 until a termination criterion is reached.

Usually the termination criterion is simply a number of iterations provided by the user. Fancy implementations stop the algorithm once some convergence has been detected.

The algorithm can be restarted with different initial centroids. The best result is kept. The quality of a result can be defined in terms of inertia: the sum of squared distances of samples to their closest cluster center. The lower the inertia, the better.

Here are a couple of intuitive demos:

- https://www.naftaliharris.com/blog/visualizing-k-means-clustering/
- https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/

## Applying this to our penguins

In [11]:
from sklearn import cluster
from sklearn import pipeline
from sklearn import preprocessing

k = 3
kmeans = cluster.KMeans(
    n_clusters=k,
    init='random',
    n_init='auto',
    algorithm='lloyd',
    random_state=42
)
kmeans.fit(features)


KMeans(init='random', n_clusters=3, n_init='auto', random_state=42)

In [15]:
kmeans.cluster_centers_.shape


(3, 4)

In [20]:
assert (kmeans.predict(X=features) == kmeans.labels_).all()


In [21]:
labels


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2,

In [13]:
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
penguins['cluster'] = labels
penguins['cluster'] = penguins['cluster'].map({
    0: 'Adelie',
    1: 'Gentoo',
    2: 'Chinstrap'
})


The most fun part/useful part is to plot the results. For every pair of features, let's plot the observations colored by the species they belong to, alongside a plot with the observations colored by the cluster they belong to. We can also highlight the cases where the species and the cluster don't match.

In [22]:
import itertools
import matplotlib.cm as cm
import matplotlib.pyplot as plt

charts = []
for feature1, feature2 in itertools.combinations(features.columns, 2):
    penguins['label_differs'] = penguins['cluster'] != penguins['species']

    charts.append(alt.hconcat(
        alt.Chart(penguins, title='Species').mark_circle().encode(
            x=feature1,
            y=feature2,
            color=alt.Color('species', scale=alt.Scale(scheme='dark2')),
            tooltip='species'
        ),
        alt.Chart(penguins, title='Clustering').mark_circle().encode(
            x=feature1,
            y=feature2,
            color=alt.Color('cluster', scale=alt.Scale(scheme='dark2')),
            tooltip='cluster'
        ),
        alt.Chart(penguins, title='Errors').mark_circle().encode(
            x=feature1,
            y=feature2,
            color=alt.Color('label_differs:N')
        )
    ))

# Combine the subplots into a single chart
combined_chart = alt.vconcat(*charts)

# Show the combined chart
combined_chart.interactive()


alt.VConcatChart(...)

It's not usually the case that we have ground truth labels to compare with. But in this case we do. This could be useful, say, for spotting outliers: penguins that are assigned to a cluster that doesn't match their species.

When the ground truth is available, a homogeneity score can be calculated. This is a measure of how well the clusters match the ground truth labels. The score is between 0 and 1, with 1 being the best possible score. The score is 1 when each cluster contains only members of a single class. The score is 0 when the clusters are completely different from the classes.

In [24]:
from sklearn import metrics

f"{metrics.homogeneity_score(species, labels):.2%}"


'80.12%'

We can get more detail with a confusion matrix.

In [25]:
species = ['Adelie', 'Gentoo', 'Chinstrap']
cm = metrics.confusion_matrix(
    penguins['species'],
    penguins['cluster'],
    labels=species
)
pd.DataFrame(cm, columns=species, index=species)


,Adelie,Gentoo,Chinstrap
Adelie,124,0,22
Gentoo,0,119,0
Chinstrap,5,0,63


The big question is how do we choose $k$? We can usually take a good guess if we can visualize the data, but that is almost never the case in practice. What we need is a metric. The idea is that we want to loop over a range of $k$ values and measure the quality of our clusters. Obviously "quality" is subject to interpretation.

One way is to try out different values for $k$ and track the intra-cluster distance: the total squared Euclidean distance from each point to the center of its cluster. This is the inertia mentioned above. The idea is that we want to choose $k$ such that the inertia is low, but not too low. If we choose $k$ too large, the inertia will be low, but the clusters will be meaningless.

In [28]:
for k in [2, 3, 4, 5, 6]:
    k_means = cluster.KMeans(n_clusters=k, init='random', n_init='auto', random_state=42)
    k_means.fit(features)
    inertia = -k_means.score(features)
    print(f'{k=} gives an inertia of {inertia:.3f}')


k=2 gives an inertia of 383.082
k=3 gives an inertia of 407.328
k=4 gives an inertia of 423.330
k=5 gives an inertia of 368.071
k=6 gives an inertia of 359.565


Another common solution is called [silhouette analysis](https://www.wikiwand.com/en/Silhouette_(clustering)), which is based on the [silhouette](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score) score. The best possible silhouette score is 1 and the worst is -1. Some good explanations and vizualizations can be found [here](https://github.com/datascience-course/2023-datascience-lectures/blob/main/22-Clustering1/22-Clustering1.ipynb). This is a fancier metric than inertia, because it takes into account the distance between clusters, and not just within clusters.

In [141]:
from sklearn import metrics

for k in [2, 3, 4, 5, 6]:
    k_means = cluster.KMeans(n_clusters=k, init='random', n_init='auto', random_state=42)
    labels = k_means.fit_predict(features)
    silhouette = metrics.silhouette_score(features, labels)
    print(f'{k=} gives a silhouette score of {silhouette:.3f}')


k=2 gives a silhouette score of 0.531
k=3 gives a silhouette score of 0.446
k=4 gives a silhouette score of 0.421
k=5 gives a silhouette score of 0.374
k=6 gives a silhouette score of 0.364
